In [1]:
import datetime
import decimal
import pydantic
import enum
from typing import *
import warnings
import QuantLib as ql
import pandas as pd
import numpy as np

import finsec as fs

import finsec.fixed_income_objs as fio

In [2]:
# fs.fixed_income_objs.Period(period='1y').model_dump()

## autoeload info:
%load_ext autoreload
%autoreload 2

In [3]:
# BusinessDayConvention.following.to_ql()
# BusinessDayConvention.from_ql(ql.Following)

In [4]:
import json
import functools
from operator import add, sub, mul

T = TypeVar('T')
ListOrT = Union[List[T], T]



In [5]:
acc1 = fio.AccrualInfo(
    start=datetime.date(2025,1,1),
    # end=datetime.date(2025,11,5),
    end='3m',
    period='1m',
    dc=fio.DayCount.Thirty360,
    front_stub_not_back=False,
)
# acc1.as_ql().dates()
print(len(acc1))
acc1.schedule().to_df()

3


,start,end,frac
0,2025-01-01,2025-02-01,0.083333
1,2025-02-01,2025-03-01,0.083333
2,2025-03-01,2025-04-01,0.083333


In [6]:
print(json.dumps(json.loads(acc1.model_dump_json()), indent=4))
fio.AccrualInfo.model_validate_json(acc1.model_dump_json())

{
    "start": "2025-01-01",
    "end": {
        "period": "3M"
    },
    "dc": "30/360",
    "freq": null,
    "cal_accrual": "null",
    "cal_pay": "null",
    "period": {
        "period": "1M"
    },
    "bdc": "F",
    "front_stub_not_back": false,
    "eom": false
}


AccrualInfo(start=datetime.date(2025, 1, 1), end=Period("3M"), dc=<DayCount.Thirty360: '30/360'>, freq=None, cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=Period("1M"), bdc=<BusinessDayConvention.following: 'F'>, front_stub_not_back=False, eom=False)

In [7]:
# class ExprOperator(enum.Enum):
#     ADD = 'ADD'
#     SUB = 'SUB'
#     MUL = 'MUL'
#     NEG = 'NEG'
#     MAX = 'MAX'
#     MIN = 'MIN'

# class ResetLookup(pydantic.BaseModel):
#     resets: Dict[Hashable, Dict[datetime.date, decimal.Decimal]]
#     @property
#     def available_containers(self)->Set[Hashable]:
#         return set(self.resets.keys())


# class RateExpression(pydantic.BaseModel):
#     def get_fixing(self, dt:datetime.date)->Optional[decimal.Decimal]:
#         pass

#     @staticmethod
#     def wrap(x:int | decimal.Decimal | ListOrT["RateExpression"])->"RateExpression":
#         if isinstance(x, decimal.Decimal):
#             return FixedRate(rate=x)
#         elif isinstance(x, int):
#             return FixedRate(rate=decimal.Decimal(x))
#         elif isinstance(x, RateExpression):
#             return x
#         else:
#             raise ValueError(f'Unknown type : {type(x)}')

#     def __add__(self, other: decimal.Decimal | ListOrT["RateExpression"]) -> "RateExpression":
#         return CompositeRateExpression( components = [self, self.wrap(other)], operator=ExprOperator.ADD)
#     def __radd__(self, other: decimal.Decimal | ListOrT["RateExpression"]) -> "RateExpression":
#         return CompositeRateExpression( components = [self, self.wrap(other)], operator=ExprOperator.ADD)

#     def __mul__(self, other: decimal.Decimal | ListOrT["RateExpression"]) -> "RateExpression":
#         return CompositeRateExpression( components = [self, self.wrap(other)], operator=ExprOperator.MUL)
#     def __rmul__(self, other: decimal.Decimal | ListOrT["RateExpression"]) -> "RateExpression":
#         return CompositeRateExpression( components = [self, self.wrap(other)], operator=ExprOperator.MUL)

#     def __sub__(self, other: decimal.Decimal | ListOrT["RateExpression"]) -> "RateExpression":
#         return self + -self.wrap(other)
#     def __rsub__(self, other: decimal.Decimal | ListOrT["RateExpression"]) -> "RateExpression":
#         return -self + self.wrap(other)

#     def __neg__(self) -> "RateExpression":
#         return self * -1
#     def __rneg__(self) -> "RateExpression":
#         return self * -1
    

    
#     @staticmethod
#     def max(*args: ListOrT["RateExpression"]) -> "RateExpression":
#         return CompositeRateExpression( components = list(args), operator=ExprOperator.MAX)
#     @staticmethod
#     def min(*args: ListOrT["RateExpression"]) -> "RateExpression":
#         return CompositeRateExpression( components = list(args), operator=ExprOperator.MIN)

#     # @property
#     # def is_constant(self):
#     #     raise NotImplementedError('child needs to instantiate constant')


# class FloatingRate(RateExpression):
#     index: Hashable
#     # reset_delay: fio.Period|None = None
#     @property
#     def is_constant(self):
#         return False

# class FixedRate(RateExpression):
#     rate: decimal.Decimal
#     is_constant:bool = True
#     def get_fixing(self, dt:datetime.date)->decimal.Decimal:
#         return self.rate

# class CompositeRateExpression(RateExpression):
#     components: List[RateExpression]
#     operator: ExprOperator

#     @property
#     def is_constant(self):
#         return all(c.is_constant for c in self.components)

#     def get_fixing(self, dt:datetime.date)->decimal.Decimal:
#         results = [c.get_fixing(dt) for c in self.components]
#         match self.operator:
#             case ExprOperator.ADD:
#                 return functools.reduce(add, results, decimal.Decimal(0))
#             case ExprOperator.SUB:
#                 if len(self.components) != 2:
#                     raise ValueError('SUB operator requires exactly 2 components.')
#                 return self.components[0].get_fixing(dt) - self.components[1].get_fixing(dt)
#             case ExprOperator.MUL:
#                 return functools.reduce(mul, results, decimal.Decimal(1))
#             case ExprOperator.MAX:
#                 return max(results)
#             case ExprOperator.MIN:
#                 return min(results)
#             case _:
#                 raise ValueError(f'Unknown operator: {self.operator}')

In [8]:
expr1 = fio.FixedRate(rate=decimal.Decimal('0.05')) * (decimal.Decimal('1')/10) + 10

expr1.get_fixing(None), expr1.is_constant

(Decimal('10.005'), True)

In [9]:
dc = ql.Actual360()

# for x in dir(dc):
#     print(x)

In [10]:
expr1.model_dump()

{'components': [{}, {}], 'operator': <ExprOperator.ADD: 'ADD'>}

In [11]:
ntnl = 1_000_000
fixleg1 = fio.Leg(
    notional=ntnl,
    cpn=fio.FixedRate(rate=decimal.Decimal('0.01')),
    acc=fio.AccrualInfo(
        start=datetime.date(2025,1,1),
        end=datetime.date(2026,1,1),
        dc=fio.DayCount.Thirty360,
        freq=12,
        bdc=fio.BusinessDayConvention.unadjusted,
    )
)
# print(fixleg1)

bnd1 = fio.Bond(
    notional=ntnl,
    leg=fixleg1,
    # settle: datetime.date
    settle_days=1
)
print(bnd1)
ql_bnd1 = bnd1.as_quantlib()

notional=Decimal('1000000') leg=Leg(notional=Decimal('1000000'), cpn=FixedRate(rate=Decimal('0.01'), is_constant=True), acc=AccrualInfo(start=datetime.date(2025, 1, 1), end=datetime.date(2026, 1, 1), dc=<DayCount.Thirty360: '30/360'>, freq=Decimal('12'), cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=None, bdc=<BusinessDayConvention.unadjusted: 'U'>, front_stub_not_back=True, eom=False), pay_delay=None) settle_days=1 credit_index=None face=100 settle=None redemption=None


In [12]:
# fio.FixedRate(rate=decimal.Decimal('0.01')).model_dump()

In [13]:
bnd1.cashflows_df().style.format({'amount':'{0:,.2f}'})

,start,end,frac,date,amount,has_occurred,rate
0,2025-01-01,2025-02-01,0.083333,2025-02-01,833.33,True,0.010000
1,2025-02-01,2025-03-01,0.083333,2025-03-01,833.33,True,0.010000
2,2025-03-01,2025-04-01,0.083333,2025-04-01,833.33,True,0.010000
3,2025-04-01,2025-05-01,0.083333,2025-05-01,833.33,True,0.010000
4,2025-05-01,2025-06-01,0.083333,2025-06-01,833.33,True,0.010000
5,2025-06-01,2025-07-01,0.083333,2025-07-01,833.33,True,0.010000
6,2025-07-01,2025-08-01,0.083333,2025-08-01,833.33,True,0.010000
7,2025-08-01,2025-09-01,0.083333,2025-09-01,833.33,True,0.010000
8,2025-09-01,2025-10-01,0.083333,2025-10-01,833.33,False,0.010000
9,2025-10-01,2025-11-01,0.083333,2025-11-01,833.33,False,0.010000


In [14]:
bnd1.model_dump()

/Users/brettg/Documents/projects/github/.venv/lib/python3.11/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `decimal` but got `int` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'notional': Decimal('1000000'),
 'leg': {'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [15]:
bnd2 = fio.Bond.model_validate_json(bnd1.model_dump_json())
bnd2.model_dump()

/Users/brettg/Documents/projects/github/.venv/lib/python3.11/site-packages/pydantic/main.py:352: UserWarning: Pydantic serializer warnings:
  Expected `decimal` but got `int` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


{'notional': Decimal('1000000'),
 'leg': {'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': Decimal('100'),
 'settle': None,
 'redemption': None}

In [16]:
bnd1.model_dump()

{'notional': Decimal('1000000'),
 'leg': {'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [ ]:
# help(ql.OvernightIndexedSwap)


Help on class OvernightIndexedSwap in module QuantLib.QuantLib:

class OvernightIndexedSwap(Swap)
 |  OvernightIndexedSwap(*args)
 |  
 |  Proxy of C++ OvernightIndexedSwap class.
 |  
 |  Method resolution order:
 |      OvernightIndexedSwap
 |      Swap
 |      Instrument
 |      LazyObject
 |      Observable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args)
 |      __init__(OvernightIndexedSwap self, Swap::Type type, Real nominal, Schedule schedule, Rate fixedRate, DayCounter fixedDC, ext::shared_ptr< OvernightIndex > const & index, Spread spread=0.0, Natural paymentLag=0, BusinessDayConvention paymentAdjustment=Following, Calendar paymentCalendar=Calendar(), bool telescopicValueDates=False, RateAveraging::Type averagingMethod=Compound) -> OvernightIndexedSwap
 |      __init__(OvernightIndexedSwap self, Swap::Type type, DoubleVector nominals, Schedule schedule, Rate fixedRate, DayCounter fixedDC, ext::shared_ptr< OvernightIndex > const & index, Sp

In [35]:
swp_ntnl = 1_000_000
swp_start = datetime.date(2025,1,1)
swp_end = '5y'
swp_index = 'SOFR'
swp = fio.Swap.make_ois(
    start=datetime.date(2025, 1, 1),
    end='5y',
    rate=3.5 / 100,
    dc_fix=fio.DayCount.Actual360,
    dc_float=fio.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index='SOFR',
    cal_pay=fio.Calendar.US_SOFR,
    notional=1_000_000,
    pay_delay=fio.Period(period='2d'),
)
display(swp.model_dump())

{'legs': ({'notional': Decimal('1000000'),
   'cpn': {'rate': Decimal('0.035'), 'is_constant': True},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
    'eom': False},
   'pay_delay': None},
  {'notional': Decimal('-1000000'),
   'cpn': {'index': 'SOFR',
    'type_': {'type_': 'overnight',
     'pay_delay': {'period': '2D'},
     'compounded_not_averaged': True}},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
  

In [38]:
swp.model_dump()

{'legs': ({'notional': Decimal('1000000'),
   'cpn': {'rate': Decimal('0.035'), 'is_constant': True},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
    'eom': False},
   'pay_delay': None},
  {'notional': Decimal('-1000000'),
   'cpn': {'index': 'SOFR',
    'type_': {'type_': 'overnight',
     'pay_delay': {'period': '2D'},
     'compounded_not_averaged': True}},
   'acc': {'start': datetime.date(2025, 1, 1),
    'end': {'period': '5Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
  

## bootstrap a treaasury curve

In [29]:
class SingleCurveConfig(pydantic.BaseModel):
    name: Hashable
    date: datetime.date
    settle_days: int
    dc: fio.DayCount
    cal: fio.Calendar

    # parallel_bump:fio.Quote = pydantic.Field(default_factory=fio.Quote)
    resets: Dict[datetime.date, decimal.Decimal] = pydantic.Field(default_factory=dict)
    helpers: Dict[str, Tuple[fio.Quote, Any]] = pydantic.Field(default_factory=dict)

    ql_curve_raw: ql.YieldTermStructure|None=None
    handle: ql.RelinkableYieldTermStructureHandle=pydantic.Field(default_factory=ql.RelinkableYieldTermStructureHandle)
    index: ql.InterestRateIndex|None=None

    class Config:
        arbitrary_types_allowed = True
    
    def benchmarks(self)->Dict[str, fio.Quote]:
        return { n:q[0] for n,q in self.helpers.items() }

    def add_helper(self, name:str, quote:fio.Quote, helper:Any):
        self.helpers[name] = (quote, helper)
    
    def add_reset(self, dt:datetime.date, rate:decimal.Decimal):
        self.resets[dt] = rate
        if self.index is not None:
            self.index.addFixing( ql.Date.from_date(dt), float(rate))
    
    def build_quantlib_index(self):
        usd = ql.USDCurrency()
        self.index = ql.OvernightIndex(
            self.name,
            self.settle_days,
            usd,
            self.cal.as_ql(),
            self.dc.as_ql,
            self.handle
        )
        self.index.addFixings(
            [ql.Date.from_date(x) for x in self.resets.keys()],
            [float(x) for x in self.resets.values()],
        )

    def build_quantlib_curve(self):
        instruments = [x[1] for x in self.helpers.values()]
        self.ql_curve_raw = ql.PiecewiseFlatForward(
            ql.Date.from_date(self.date),
            instruments,
            self.dc.as_ql,
        )
        self.handle.linkTo(self.ql_curve_raw)
    
    # def plot_forwards()

# class CurveInterp(pydantic.BaseModel):
#     order: List[Literal[0,1,2]]
#     joint_date:
    



In [22]:
tsy_curve = SingleCurveConfig(
    name='USGOVT',
    date=datetime.date(2025, 1, 1),
    settle_days=1,
    dc=fio.DayCount.ActualActual,
    cal=fio.Calendar.US_GovernmentBond,
)

In [30]:
bond_tenors = ['1y','2y','5y','10y']
bonds = [
    fio.Bond(
        credit_index='USGOVT',
        notional=ntnl,
        leg=fio.Leg(
            notional=100,
            cpn=fio.FixedRate(rate=5.0 / 100),
            acc=fio.AccrualInfo(
                start=datetime.date(2025,1,1),
                end=ten,
                dc=fio.DayCount.ActualActual,
                freq=2,
                bdc=fio.BusinessDayConvention.modified_following,
            )
        ),
        settle_days=1
    )
    for ten in bond_tenors
]

# bond_prices = [100] * len(bond_tenors)
bond_prices = [
    100,
    99,
    98,
    97,
]
bond_helpers = [bnd.as_quantlib_helper() for bnd in bonds]
for px, q in zip(bond_prices, bond_helpers):
    q[0].setValue(px)

In [31]:
for ten, qh in zip(bond_tenors, bond_helpers):
    q,h = qh
    tsy_curve.add_helper( ten, q, h)

In [32]:
tsy_curve.build_quantlib_curve()
tsy_curve.ql_curve_raw.enableExtrapolation()

In [33]:
# tsy_curve.plot(datetime.date(2025,1,1), datetime.date(2035,1,1))

In [34]:
# swp_crv0_ql = ql.PiecewiseFlatForward(
# )

# help(ql.PiecewiseFlatForward)
# PiecewiseFlatForward self
#     Date referenceDate
#     RateHelperVector instruments
#     DayCounter dayCounter
#     QuoteHandleVector jumps=std::vector< Handle< Quote > >()
#     DateVector jumpDates=std::vector< Date >()
#     BackwardFlat i=BackwardFlat()
#     IterativeBootstrap b=_IterativeBootstrap()

for x in dir(ql):
    if 'piecewise' in x.lower():
        print(x)

PiecewiseConstantCorrelation
PiecewiseConstantParameter
PiecewiseConvexMonotoneZero
PiecewiseCubicZero
PiecewiseFlatForward
PiecewiseFlatHazardRate
PiecewiseKrugerLogDiscount
PiecewiseKrugerZero
PiecewiseLinearForward
PiecewiseLinearZero
PiecewiseLogCubicDiscount
PiecewiseLogLinearDiscount
PiecewiseLogMixedLinearCubicDiscount
PiecewiseNaturalCubicZero
PiecewiseNaturalLogCubicDiscount
PiecewiseSplineCubicDiscount
PiecewiseTimeDependentHestonModel
PiecewiseYoYInflation
PiecewiseZeroInflation
PiecewiseZeroSpreadedTermStructure


In [ ]:
# help(ql)